# **Import Google Drive**


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Setting**


In [3]:
from sklearn.isotonic import isotonic_regression
import cvxpy as cp
import numpy as np
import csv, os
from scipy import stats
from sklearn import metrics
import itertools
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import statistics
from scipy.stats import levene
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

from statsmodels.stats.contingency_tables import mcnemar


# **Load CSV file into a pandas DataFrame**

In [ ]:
# Load CSV file into a pandas DataFrame
df = pd.read_csv(r'rank_author_dec.csv')
df = df.drop_duplicates(['submission_id', 'author_id'])


# **Highest-ranked vs Lowest-ranked in Final Decisions**



In [ ]:
from sklearn.isotonic import isotonic_regression
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


# Load CSV file into a pandas DataFrame
df_rank = pd.read_csv(r'aggregated_results_anonymized.csv')

authors = df_rank['author_id'].unique()

high_rank_submissions = []
low_rank_submissions = []
for author in authors:
  rank = df_rank[df_rank['author_id'] == author]['rank_1'].tolist()
  high_rank_submissions.append( df_rank[(df_rank['author_id'] == author) & (df_rank['rank_1'] == min(rank))]['submission_id'].tolist()[0] )
  low_rank_submissions.append( df_rank[(df_rank['author_id'] == author) & (df_rank['rank_1'] == max(rank))]['submission_id'].tolist()[0] )


oral = 0
accept = 0
reject = 0
withdraw = 0
for submission in high_rank_submissions:
  if df_rank[df_rank['submission_id'] == submission]['decision'].tolist()[0] == 'Accept (Oral & Poster)':
    oral += 1
  if df_rank[df_rank['submission_id'] == submission]['decision'].tolist()[0] == 'Accept (Poster)':
    accept += 1
  if df_rank[df_rank['submission_id'] == submission]['decision'].tolist()[0] == 'Reject':
    reject += 1
  if df_rank[df_rank['submission_id'] == submission]['decision'].tolist()[0] == 'Withdrawn or Desk Rejected':
    withdraw += 1

print("High-ranked:", round(oral/len(high_rank_submissions),5), round(accept/len(high_rank_submissions),5), round(reject/len(high_rank_submissions),5), round(withdraw/len(high_rank_submissions),5) )

oral = 0
accept = 0
reject = 0
withdraw = 0
for submission in low_rank_submissions:
  if df_rank[df_rank['submission_id'] == submission]['decision'].tolist()[0] == 'Accept (Oral & Poster)':
    oral += 1
  if df_rank[df_rank['submission_id'] == submission]['decision'].tolist()[0] == 'Accept (Poster)':
    accept += 1
  if df_rank[df_rank['submission_id'] == submission]['decision'].tolist()[0] == 'Reject':
    reject += 1
  if df_rank[df_rank['submission_id'] == submission]['decision'].tolist()[0] == 'Withdrawn or Desk Rejected':
    withdraw += 1

print("Low-ranked:", round(oral/len(low_rank_submissions),5), round(accept/len(low_rank_submissions),5), round(reject/len(low_rank_submissions),5), round(withdraw/len(low_rank_submissions),5) )


High-ranked: 0.04658 0.33509 0.52817 0.09016
Low-ranked: 0.02329 0.23366 0.57776 0.16529


# **Perform Chi Squared Test**


In [6]:

## Organize all the submissions by {author: [submission, rank]}.
author_submission_rank_old = {}
authors = df['author_id'].unique()

for author in authors:
    author_submission_rank_old[author] = []
    submissions = df[df['author_id'] == author]['submission_id'].tolist()
    for i in range(len(submissions)):
        rank = df[(df['submission_id'] == submissions[i]) & (df['author_id'] == author)]['rank'].tolist()[0]
        author_submission_rank_old[author].append((submissions[i], rank))


# Sort submissions by rank.
def sort_submissions(author_submission_rank_old):
    for author in author_submission_rank_old:
        author_submission_rank_old[author].sort(key=lambda x: x[1], reverse=False)
    return author_submission_rank_old

author_submission_rank_old = sort_submissions(author_submission_rank_old)


# If final decesion = xxx, record 1; Otherwise, record 0.
df['decision'] = df['decision'].map({'Withdrawn or Desk Rejected': 1.0, 'Reject': 0.0, 'Accept (Poster)': 0.0, 'Accept (Oral & Poster)':0.0})

# Append highest-ranked submission into High_decision list; Append lowest-ranked submission into Low_decision list.
High_decision = []
Low_decision = []
for author in author_submission_rank_old:
        decision = df[df['submission_id'] == author_submission_rank_old[author][0][0]]['decision'].tolist()
        decision = decision[0]
        High_decision.append(decision)

        decision = df[df['submission_id'] == author_submission_rank_old[author][-1][0]]['decision'].tolist()
        decision = decision[0]
        Low_decision.append(decision)


In [7]:
contingency_table = [[0, 0], [0, 0]]
for i in range(len(High_decision)):
    if High_decision[i] == 1:
        contingency_table[0][0] += 1
    if High_decision[i] == 0:
        contingency_table[0][1] += 1
    if Low_decision[i] == 1:
        contingency_table[1][0] += 1
    if Low_decision[i] == 0:
        contingency_table[1][1] += 1


res = stats.chi2_contingency(np.array(contingency_table), correction=True, lambda_=None)

result = mcnemar(contingency_table, exact=False)  # exact=True for small sample sizes

print("high-ranked submissions:", contingency_table[0][0]/(contingency_table[0][0] + contingency_table[0][1]))
print("low-ranked submissions:", contingency_table[1][0]/(contingency_table[1][0] + contingency_table[1][1]))

print("Chi-square test statistic:", res.statistic)
print("P-value:", res.pvalue)

high-ranked submissions: 0.09016393442622951
low-ranked submissions: 0.16244411326378538
Chi-square test statistic: 31.115904673849464
P-value: 2.4307155692960957e-08
